In [1]:
#modul de carrega i de neteja de les dades, comú des de la primera pràctica

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#per realitzar les grafiques de regrassió ha estat necessari ampliar els chunks k

plt.rcParams["agg.path.chunksize"]=100000

#carreguem els 3 camps en un de sol per poder utilitzar-el als analisi
#df = pd.read_csv("DelayedFlights.csv", parse_dates= {"Data" : ["Year","Month","DayofMonth"]})
df1 = pd.read_csv("DelayedFlights.csv")


msk = np.random.rand(len(df1)) < 0.3

df = df1[msk]

#millorat el procés de neteja / analisi de la qualitat en les dades
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


df["ArrDelay"] = df['ArrDelay'].fillna(df['ArrDelay'].mean())

df["DepDelay"] = df['DepDelay'].fillna(df['DepDelay'].mean())

df["AirTime"] = df['AirTime'].fillna(df['AirTime'].mean())
df["Distance"] = df['Distance'].fillna(df['Distance'].mean())
df["CarrierDelay"] = df["CarrierDelay"].fillna(0)
df["WeatherDelay"] = df["WeatherDelay"].fillna(0)
df["NASDelay"] = df["NASDelay"].fillna(0)
df["SecurityDelay"] = df["SecurityDelay"].fillna(0)
df["LateAircraftDelay"] = df["LateAircraftDelay"].fillna(0)
df["CarrierDelay"] = df["CarrierDelay"].fillna(0)

missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


df.dropna(inplace = True)

#esborren totes les files que segueixin contenint algun null, tot i la perdua dinformació
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

#bloc per transformar les dades descriptives en valors numerics
# i ajustar les escales


#transformo les dades origen i destí

ordinal_encoder = OrdinalEncoder()
df["Origin"] = ordinal_encoder.fit_transform( df[['Origin']])


df["Dest"] = ordinal_encoder.fit_transform(df[['Dest']])

df["Retard"] = df["ArrDelay"] > 0


#df["UniqueCarrier"] = ordinal_encoder.fit_transform(df[['UniqueCarrier']])

<ipython-input-1-77e854c75d4b>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ArrDelay"] = df['ArrDelay'].fillna(df['ArrDelay'].mean())
<ipython-input-1-77e854c75d4b>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DepDelay"] = df['DepDelay'].fillna(df['DepDelay'].mean())
<ipython-input-1-77e854c75d4b>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [2]:
df2_x = pd.DataFrame(np.c_[df['AirTime']], columns = ['AirTime'])

df2_y = pd.DataFrame(df.ArrDelay)

x_train, x_test, y_train, y_test = train_test_split(df2_x, df2_y, test_size=0.33, random_state=42)


In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
df.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Retard
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0,True
14,22,2008,1,3,4,1900.0,1840,1956.0,1950,WN,...,5.0,0,N,0,0.0,0.0,0.0,0.0,0.0,True
16,25,2008,1,3,4,1520.0,1455,1619.0,1605,WN,...,7.0,0,N,0,0.0,0.0,0.0,0.0,0.0,True
17,26,2008,1,3,4,1422.0,1255,1657.0,1610,WN,...,6.0,0,N,0,40.0,0.0,0.0,0.0,7.0,True
22,35,2008,1,3,4,1634.0,1555,1859.0,1845,WN,...,6.0,0,N,0,0.0,0.0,0.0,0.0,0.0,True


In [4]:
#Crea almenys tres models de classificació diferents per intentar predir el millor possible
#l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.
#Considera si el vol ha arribat tard o no (ArrDelay > 0).
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

#split dataset in features and target variable
feature_cols = ["Year","Month","DayofMonth","DayOfWeek","DepDelay"]
X = df[feature_cols] # Features
y = df.Retard # Target variable
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Precisió arbre classificació per defecte criteri gini:",metrics.accuracy_score(y_test, y_pred))

print("Total valors y_test",y_test.value_counts ())

print("Mitjana valors y_test", y_test.mean())

# calculate the percentage of zeros
print("Percentatges de 0s a y_Test", 1 - y_test.mean())


# calculate null accuracy in a single line of code
# only for binary classification problems coded as 0/1
print("Diferència maxima de la precisió",max(y_test.mean(), 1 - y_test.mean()))


# calculate null accuracy (for multi-class classification problems)
print("Precisió nula", y_test.value_counts().head(1) / len(y_test))

# print the first 25 true and predicted responses
print('True:', y_test.values[0:25])
print('False:', y_pred[0:25])

# IMPORTANT: first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
print(metrics.confusion_matrix(y_test, y_pred))


# print the first 25 true and predicted responses
print('True', y_test.values[0:25])
print('Pred', y_pred[0:25])

# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

# use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print("Precisió de la classificació: " , metrics.accuracy_score(y_test, y_pred))

classification_error = (FP + FN) / float(TP + TN + FP + FN)

print("Error de classificació:", classification_error)
print(1 - metrics.accuracy_score(y_test, y_pred))

sensitivity = TP / float(FN + TP)

print("Sensibilitat: quan el valor real és positiu, amb quina freqüència és correcta la predicció", sensitivity)
print(metrics.recall_score(y_test, y_pred))

specificity = TN / (TN + FP)



print("Especificitat: quan el valor real és negatiu, amb quina freqüència és correcta la predicció",specificity)


false_positive_rate = FP / float(TN + FP)

print("Taxa de fals positius: quan el valor real és negatiu, amb quina freqüència és incorrecta la predicció",false_positive_rate)
print(1 - specificity)


precision = TP / float(TP + FP)

print("Precisió: quan es prediu un valor positiu, amb quina freqüència és correcta la predicció",precision)
print(metrics.precision_score(y_test, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))



Precisió arbre classificació per defecte criteri gini: 0.8915366364754169
Total valors y_test True     154785
False     18444
Name: Retard, dtype: int64
Mitjana valors y_test 0.8935282198708069
Percentatges de 0s a y_Test 0.10647178012919312
Diferència maxima de la precisió 0.8935282198708069
Precisió nula True    0.893528
Name: Retard, dtype: float64
True: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False]
False: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]
[[  1086  17358]
 [  1431 153354]]
True [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False]
Pred [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  Tru

In [5]:

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

# instantiate model
logreg = LogisticRegression()

# fit model
logreg.fit(X_train, y_train)

y_pred_class = logreg.predict(X_test)

print("Precisió regressió logistica", metrics.accuracy_score(y_test, y_pred_class))


# print the first 10 predicted responses
# 1D array (vector) of binary values (0, 1)
logreg.predict(X_test)[0:10]

# print the first 10 predicted probabilities of class membership
logreg.predict_proba(X_test)[0:10]

# print the first 10 predicted probabilities for class 1
logreg.predict_proba(X_test)[0:10, 1]

# store the predicted probabilities for class 1
y_pred_prob = logreg.predict_proba(X_test)[:, 1]

# allow plots to appear in the notebook

# IMPORTANT: first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
print(metrics.confusion_matrix(y_test, y_pred))


# print the first 25 true and predicted responses
print('True', y_test.values[0:25])
print('Pred', y_pred[0:25])

# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

# use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print("Precisió de la classificació: " , metrics.accuracy_score(y_test, y_pred))

classification_error = (FP + FN) / float(TP + TN + FP + FN)

print("Error de classificació:", classification_error)
print(1 - metrics.accuracy_score(y_test, y_pred))

sensitivity = TP / float(FN + TP)

print("Sensibilitat: quan el valor real és positiu, amb quina freqüència és correcta la predicció", sensitivity)
print(metrics.recall_score(y_test, y_pred))

specificity = TN / (TN + FP)



print("Especificitat: quan el valor real és negatiu, amb quina freqüència és correcta la predicció",specificity)


false_positive_rate = FP / float(TN + FP)

print("Taxa de fals positius: quan el valor real és negatiu, amb quina freqüència és incorrecta la predicció",false_positive_rate)
print(1 - specificity)


precision = TP / float(TP + FP)

print("Precisió: quan es prediu un valor positiu, amb quina freqüència és correcta la predicció",precision)
print(metrics.precision_score(y_test, y_pred))


from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))



Precisió regressió logistica 0.8935282198708069
[[  1086  17358]
 [  1431 153354]]
True [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False]
Pred [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]
[[  1086  17358]
 [  1431 153354]]
0.8915366364754169
Precisió de la classificació:  0.8915366364754169
Error de classificació: 0.10846336352458306
0.10846336352458308
Sensibilitat: quan el valor real és positiu, amb quina freqüència és correcta la predicció 0.9907549181122202
0.9907549181122202
Especificitat: quan el valor real és negatiu, amb quina freqüència és correcta la predicció 0.058880936890045546
Taxa de fals positius: quan el valor real és negatiu, amb quina freqüència és incorrecta la predicció 0.9411190631099544
0.9411190631099544
Precisió: quan es prediu un valor positiu, amb qu

In [6]:
#K-Nearest Neighbors
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


# IMPORTANT: first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
print(metrics.confusion_matrix(y_test, y_pred))


# print the first 25 true and predicted responses
print('True', y_test.values[0:25])
print('Pred', y_pred[0:25])

# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

# use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print("Precisió de la classificació: " , metrics.accuracy_score(y_test, y_pred))

classification_error = (FP + FN) / float(TP + TN + FP + FN)

print("Error de classificació:", classification_error)
print(1 - metrics.accuracy_score(y_test, y_pred))

sensitivity = TP / float(FN + TP)

print("Sensibilitat: quan el valor real és positiu, amb quina freqüència és correcta la predicció", sensitivity)
print(metrics.recall_score(y_test, y_pred))

specificity = TN / (TN + FP)



print("Especificitat: quan el valor real és negatiu, amb quina freqüència és correcta la predicció",specificity)


false_positive_rate = FP / float(TN + FP)

print("Taxa de fals positius: quan el valor real és negatiu, amb quina freqüència és incorrecta la predicció",false_positive_rate)
print(1 - specificity)


precision = TP / float(TP + FP)

print("Precisió: quan es prediu un valor positiu, amb quina freqüència és correcta la predicció",precision)
print(metrics.precision_score(y_test, y_pred))



from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


[[  3249  15195]
 [  5887 148898]]
              precision    recall  f1-score   support

       False       0.36      0.18      0.24     18444
        True       0.91      0.96      0.93    154785

    accuracy                           0.88    173229
   macro avg       0.63      0.57      0.58    173229
weighted avg       0.85      0.88      0.86    173229

[[  3249  15195]
 [  5887 148898]]
True [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False]
Pred [ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True]
[[  3249  15195]
 [  5887 148898]]
0.8782998227779414
Precisió de la classificació:  0.8782998227779414
Error de classificació: 0.12170017722205866
0.12170017722205861
Sensibilitat: quan el valor real és positiu, amb quina freqüència és correcta la predicció 0.9619665988306361
0.96196

In [7]:
#XGBoost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

cv_results.head()
print((cv_results["test-rmse-mean"]).tail(1))

xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)


# IMPORTANT: first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
print(metrics.confusion_matrix(y_test, y_pred))


# print the first 25 true and predicted responses
print('True', y_test.values[0:25])
print('Pred', y_pred[0:25])

# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

# use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print("Precisió de la classificació: " , metrics.accuracy_score(y_test, y_pred))

classification_error = (FP + FN) / float(TP + TN + FP + FN)

print("Error de classificació:", classification_error)
print(1 - metrics.accuracy_score(y_test, y_pred))

sensitivity = TP / float(FN + TP)

print("Sensibilitat: quan el valor real és positiu, amb quina freqüència és correcta la predicció", sensitivity)
print(metrics.recall_score(y_test, y_pred))

specificity = TN / (TN + FP)



print("Especificitat: quan el valor real és negatiu, amb quina freqüència és correcta la predicció",specificity)


false_positive_rate = FP / float(TN + FP)

print("Taxa de fals positius: quan el valor real és negatiu, amb quina freqüència és incorrecta la predicció",false_positive_rate)
print(1 - specificity)


precision = TP / float(TP + FP)

print("Precisió: quan es prediu un valor positiu, amb quina freqüència és correcta la predicció",precision)
print(metrics.precision_score(y_test, y_pred))



from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


[23:05:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 0.328209
[23:05:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[23:05:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[23:05:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
49    0.284557
Name: test-rmse-mean, dtype: float64
[23:05:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[[  3249  15195]
 [  5887 148898]]
True [ True  